In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [ ]:
llm = ChatOllama(
    model=MODEL, 
    temperature=TEMPERATURE,
    # base_url="http://host.docker.internal:11434", # if running in the studio
    ).bind_tools(tools)


class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    claim: str
    evidence: list[dict]  # {'name': tool name, 'args': {kwargs}, 'result': str}

In [3]:
# First, load the autoreload extension
%load_ext autoreload

# Set autoreload to automatically reload all modules
%autoreload 2

In [4]:
# from dotenv import load_dotenv
import importlib
import os
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage, SystemMessage
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Annotated, TypedDict


MODEL = "gpt-3.5-turbo-0125"
TEMPERATURE = 0
# load_dotenv('../.env', override=True)

In [34]:
import requests
from langchain.agents import Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Define a lookup function that interacts with the external API
def remote_api_look_up(query: str, api_key: str, url: str):
    # Construct the API query (example using New York Times API)
    response = requests.get(f"{url}?q={query}&api-key={api_key}")
    
    if response.status_code == 200:
        return response.json()  # Return the JSON data from the API response
    else:
        return f"Error: {response.status_code}"


# Initialize the tool that will call the lookup function
def api_lookup_tool(query: str):
    # Set your New York Times API details
    url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
    api_key = "YOUR_API_KEY_HERE"  # Replace with your actual API key
    
    # Call the lookup function
    return nyt_lookup(query, api_key, url)


tool = Tool(
        name="NYT_Search_Tool",
        func=api_lookup_tool,
        description="Searches the New York Times API for articles."
    )
from tools.calculator import add, divide
add = Tool(name = "add", func = add, description="Add `a` and `b`")
divide = Tool(name = "devide", func =divide, description="Divide `a` and `b`")
tools = [tool,add,divide]

In [14]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname('__file__'), '../..')))
from core.agents.tools.remote_tool import remote_tool_func
from langchain.agents import Tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain.agents import AgentExecutor, create_openai_tools_agent

tool = Tool(
    name="NewsAPI",
    func=remote_tool_func,
    description="Get information about a news article from a URL"
)

tools = [tool]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the tools provided to answer the user's question."),
    ("human", "{input}"),
    ("ai", "{agent_scratchpad}")
])

model = ChatOllama(model="mistral-nemo", temperature=0)

agent = create_openai_tools_agent(model, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "Can you get information about a news article at https://example.com/news-article?"})

print(f"Agent result: {result}")



> Entering new AgentExecutor chain...


> Finished chain.
Agent result: {'input': 'Can you get information about a news article at https://example.com/news-article?', 'output': ''}


In [35]:
llm = ChatOpenAI(
    model=MODEL, 
    temperature=TEMPERATURE,
    # base_url="http://host.docker.internal:11434", # if running in the studio
    ).bind_tools(tools)

In [21]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    claim: str
    evidence: list[dict]  # {'name': tool name, 'args': {kwargs}, 'result': str}

In [28]:
from IPython.display import Image, display



with open('prompts/research_agent_system_prompt.txt', 'r') as f:
    sys_msg = SystemMessage(content=f.read())

def preprocessing(state: State):
    """
    Preprocesses state before sending to the assistant for tool routing.
    Currently, this just extracts the claim from the state and sets it as a HumanMessage
    following the SystemMessage
    """
    return {"messages": HumanMessage(content=state['claim'])}

def assistant(state: State) -> State:
    response = llm.invoke(state['messages'])
    return {"messages": response}

def postprocessing(state: State) -> State:
    """
    Scan the message history to extract tool calls and results into tuples:
    (tool_name, tool_args, tool_result) for the 'evidence' list in the state
    """
    
    evidence = []
    for i in range(len(state['messages'])):
        message = state['messages'][i]
        if isinstance(message, AIMessage) and hasattr(message, 'tool_calls'):
            for tool_call in message.tool_calls:
                # Scan later messages for the corresponding ToolMessage
                for j in range(i + 1, len(state['messages'])):
                    next_message = state['messages'][j]
                    if isinstance(next_message, ToolMessage) and next_message.tool_call_id == tool_call['id']:
                        # Found the corresponding ToolMessage
                        evidence.append({
                            'name': tool_call['name'], 
                            'args': tool_call['args'], 
                            'result': next_message.content})
                        break

    return {'evidence': evidence}
    # return state


# Graph
builder = StateGraph(State)

# Define nodes: these do the work
builder.add_node("preprocessing", preprocessing)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_node("postprocessing", postprocessing)

# Define edges: these determine how the control flow moves
builder.add_edge(START, "preprocessing")
builder.add_edge("preprocessing", "assistant")
builder.add_conditional_edges(
    source="assistant",
    path=tools_condition,
    path_map={'tools': 'tools', '__end__': 'postprocessing'}
)
builder.add_edge("tools", "assistant")
builder.add_edge("postprocessing", END)

react_graph = builder.compile()

# Show
# display(Image(react_graph.get_graph(xray=False).draw_mermaid_png()))

In [5]:
# from langchain_core.messages import HumanMessage, SystemMessage
claim = "1/3 is bigger than 1/4."
# initial_state = {"claim": claim}
# final_state = graph.invoke(initial_state)

messages = [sys_msg]
messages = react_graph.invoke({"messages": messages, "claim": claim})

In [6]:
for m in messages['messages']:
    m.pretty_print()

================================ System Message ================================

You are an **Evidence Retrieval Agent**. Your role is to retrieve supporting or contradictory evidence for a given claim using the tools available to you.  

### **Rules:**  
1. **Do Not Evaluate the Claim** – Your task is to retrieve evidence, not to judge the claim's truth.  
2. **Select and Use Tools Strategically** – Based on the claim, think carefully which tools would help, and construct the correct tool calls to retrieve relevant evidence.  
3. **Return Only Tool Messages** – Your output should consist solely of tool calls with properly structured arguments. Do not generate natural language explanations.  
4. **Handle Ambiguity Thoughtfully** – If the claim is unclear, attempt a best-guess search or request clarification through a tool call if applicable.  
5. **Retrieve Evidence from Multiple Perspectives** – If conflicting evidence exists, return both supporting and contradictory sources.  

### 

In [7]:
from pprint import pprint
pprint(messages)

{'claim': '1/3 is bigger than 1/4.',
 'evidence': [{'args': {'query': 'Compare the sizes of fractions 1/3 and 1/4.'},
               'name': 'query',
               'result': "No Wikipedia page found for 'Compare the sizes of "
                         "fractions 1/3 and 1/4.'."}],
 'messages': [SystemMessage(content="You are an **Evidence Retrieval Agent**. Your role is to retrieve supporting or contradictory evidence for a given claim using the tools available to you.  \n\n### **Rules:**  \n1. **Do Not Evaluate the Claim** – Your task is to retrieve evidence, not to judge the claim's truth.  \n2. **Select and Use Tools Strategically** – Based on the claim, think carefully which tools would help, and construct the correct tool calls to retrieve relevant evidence.  \n3. **Return Only Tool Messages** – Your output should consist solely of tool calls with properly structured arguments. Do not generate natural language explanations.  \n4. **Handle Ambiguity Thoughtfully** – If the claim i

ToolMessage(content="No Wikipedia page found for 'Compare the sizes of fractions 1/3 and 1/4.'.", name='query', id='dd7e1db8-9f8d-412e-96d6-bf28b3aec30b', tool_call_id='e5d05507-9826-4647-9c62-0485a24e75c5')

In [14]:
response = react_graph.nodes['assistant'].invoke(
    {"messages": [HumanMessage(content="hi there")]}
)

In [15]:
print(response)

{'messages': AIMessage(content="Hello! How can I help you today? Let's have a nice conversation. If you need any calculations, I can do that too! 😊", additional_kwargs={}, response_metadata={'model': 'mistral-nemo', 'created_at': '2025-03-15T20:58:08.533286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4515353959, 'load_duration': 617177417, 'prompt_eval_count': 263, 'prompt_eval_duration': 3137000000, 'eval_count': 32, 'eval_duration': 759000000, 'message': Message(role='assistant', content="Hello! How can I help you today? Let's have a nice conversation. If you need any calculations, I can do that too! 😊", images=None, tool_calls=None)}, id='run-5b14021e-fbe7-4626-ba5c-aee5d59ba529-0', usage_metadata={'input_tokens': 263, 'output_tokens': 32, 'total_tokens': 295})}
